# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 11:10AM,239547,10,0085766241,ISDIN Corporation,Released
1,Jun 14 2022 11:09AM,239546,10,0085766249,ISDIN Corporation,Released
2,Jun 14 2022 11:02AM,239545,10,MET211216,"Methapharm, Inc.",Released
3,Jun 14 2022 9:57AM,239544,12,617801,"NBTY Global, Inc.",Released
4,Jun 14 2022 9:57AM,239544,12,617802,"NBTY Global, Inc.",Released
5,Jun 14 2022 9:53AM,239543,10,SO94275-June16th,"Senseonics, Incorporated",Released
6,Jun 14 2022 9:49AM,239522,15,VT80173719,"Virtus Pharmaceuticals, LLC",Released
7,Jun 14 2022 9:49AM,239522,15,VT80173720,"Virtus Pharmaceuticals, LLC",Released
8,Jun 14 2022 9:49AM,239522,15,VT80173721,"Virtus Pharmaceuticals, LLC",Released
9,Jun 14 2022 9:49AM,239522,15,VT80173722,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,239543,Released,1
33,239544,Released,2
34,239545,Released,1
35,239546,Released,1
36,239547,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239543,NaN,NaN,1.0
239544,NaN,NaN,2.0
239545,NaN,NaN,1.0
239546,NaN,NaN,1.0
239547,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239463,25.0,1.0,3.0
239473,0.0,1.0,0.0
239479,0.0,0.0,33.0
239480,17.0,4.0,0.0
239481,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239463,25,1,3
239473,0,1,0
239479,0,0,33
239480,17,4,0
239481,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239463,25,1,3
1,239473,0,1,0
2,239479,0,0,33
3,239480,17,4,0
4,239481,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239463,25,1,3
1,239473,,1,
2,239479,,,33
3,239480,17,4,
4,239481,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation
1,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation
2,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc."
3,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc."
5,Jun 14 2022 9:53AM,239543,10,"Senseonics, Incorporated"
6,Jun 14 2022 9:49AM,239522,15,"Virtus Pharmaceuticals, LLC"
26,Jun 14 2022 9:46AM,239538,15,"Brookfield Pharmaceuticals, LLC"
52,Jun 14 2022 9:46AM,239539,15,"Carwin Pharmaceutical Associates, LLC"
55,Jun 14 2022 9:37AM,239537,15,"Laser Pharmaceuticals, LLC"
75,Jun 14 2022 9:22AM,239535,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,,,1
1,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,,,1
2,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",,,1
3,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",,,2
4,Jun 14 2022 9:53AM,239543,10,"Senseonics, Incorporated",,,1
5,Jun 14 2022 9:49AM,239522,15,"Virtus Pharmaceuticals, LLC",,,20
6,Jun 14 2022 9:46AM,239538,15,"Brookfield Pharmaceuticals, LLC",,,26
7,Jun 14 2022 9:46AM,239539,15,"Carwin Pharmaceutical Associates, LLC",,,3
8,Jun 14 2022 9:37AM,239537,15,"Laser Pharmaceuticals, LLC",,,20
9,Jun 14 2022 9:22AM,239535,20,Alumier Labs Inc.,,7,23


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,1,,
1,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,1,,
2,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",1,,
3,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",2,,
4,Jun 14 2022 9:53AM,239543,10,"Senseonics, Incorporated",1,,
5,Jun 14 2022 9:49AM,239522,15,"Virtus Pharmaceuticals, LLC",20,,
6,Jun 14 2022 9:46AM,239538,15,"Brookfield Pharmaceuticals, LLC",26,,
7,Jun 14 2022 9:46AM,239539,15,"Carwin Pharmaceutical Associates, LLC",3,,
8,Jun 14 2022 9:37AM,239537,15,"Laser Pharmaceuticals, LLC",20,,
9,Jun 14 2022 9:22AM,239535,20,Alumier Labs Inc.,23,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,1,,
1,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,1,,
2,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",1,,
3,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",2,,
4,Jun 14 2022 9:53AM,239543,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,1.0,NaN,NaN
1,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,1.0,NaN,NaN
2,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",2.0,NaN,NaN
4,Jun 14 2022 9:53AM,239543,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 11:10AM,239547,10,ISDIN Corporation,1.0,0.0,0.0
1,Jun 14 2022 11:09AM,239546,10,ISDIN Corporation,1.0,0.0,0.0
2,Jun 14 2022 11:02AM,239545,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Jun 14 2022 9:57AM,239544,12,"NBTY Global, Inc.",2.0,0.0,0.0
4,Jun 14 2022 9:53AM,239543,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4071864,231.0,8.0,19.0
12,239544,2.0,0.0,0.0
15,958136,69.0,0.0,0.0
19,478963,1.0,1.0,0.0
20,478998,26.0,8.0,25.0
21,718448,3.0,0.0,0.0
22,239497,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4071864,231.0,8.0,19.0
1,12,239544,2.0,0.0,0.0
2,15,958136,69.0,0.0,0.0
3,19,478963,1.0,1.0,0.0
4,20,478998,26.0,8.0,25.0
5,21,718448,3.0,0.0,0.0
6,22,239497,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,231.0,8.0,19.0
1,12,2.0,0.0,0.0
2,15,69.0,0.0,0.0
3,19,1.0,1.0,0.0
4,20,26.0,8.0,25.0
5,21,3.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,231.0
1,12,Released,2.0
2,15,Released,69.0
3,19,Released,1.0
4,20,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,19.0,0.0,0.0,0.0,25.0,0.0,0.0
Executing,8.0,0.0,0.0,1.0,8.0,0.0,0.0
Released,231.0,2.0,69.0,1.0,26.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,19.0,0.0,0.0,0.0,25.0,0.0,0.0
1,Executing,8.0,0.0,0.0,1.0,8.0,0.0,0.0
2,Released,231.0,2.0,69.0,1.0,26.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,19.0,0.0,0.0,0.0,25.0,0.0,0.0
1,Executing,8.0,0.0,0.0,1.0,8.0,0.0,0.0
2,Released,231.0,2.0,69.0,1.0,26.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()